In [6]:
# This notebook creates the final noise tresholds 
# for use in Spark streaming.
import glob
import pandas
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import signal
import os

raw_filenames = sorted(glob.glob('data/#2_all_noise_segments/*.csv'))
def find_index_of_list_containing_string(l, s):
    for e in l:
        if s in e:
            return l.index(e)
file_to_import = raw_filenames[find_index_of_list_containing_string(raw_filenames, "part")]
# load the header
file_header = np.array(pandas.read_csv("data/experiments_raw_csv_file_header.txt", header=None))[0]
# Load the 61 column file
raw_file = pandas.read_csv(file_to_import, names=file_header)

print("Minutes of extracted noise on each electrode: " + str(raw_file.count()[0]/(10000*60)) + str(" [m]"))

#for electrode in [file_header[find_index_of_list_containing_string(list(file_header), "87")]]: 
for electrode in file_header[1:]:

    print("select electrode: " + electrode)
    
    S = np.array(raw_file[electrode])
    # Sampling
    Fs = 10000 # [1/s] # 10 kHz
    dt = 1/float(Fs) # [s]
    L = len(S) # [#samples]

    t = np.arange(0,L*dt/60,dt/60)
    """
    plt.figure()
    plt.title(electrode + " combined noise segments")
    plt.xlabel("Time [m]")
    plt.ylabel("Voltage [pV]")
    plt.plot(t,S)
    """
    # Power spectral density of first 1250 segments
    f_fft_scipy, Sxx_fft_scipy = signal.periodogram(S[0:1250], fs=Fs, scaling="density")
    """
    plt.figure()
    plt.grid()
    plt.title(electrode + " PSD of first 1250 noise segments")
    plt.xlabel("Hz [1/s]")
    plt.ylabel("PSD [pV^2 / Hz]")
    plt.plot(f_fft_scipy, Sxx_fft_scipy)
    """
    # Average PSD of all 80 % overlapping 1250 segments
    total_samples_of_noise = len(S)
    window_start_sample = 0
    window_duration_samples = 1250
    window_overlap = 0.8
    window_offsets = np.arange(0, total_samples_of_noise - window_duration_samples, \
                               int(np.ceil(window_duration_samples*(1-window_overlap))))
    # prepare PSD matrix
    number_of_bins = f_fft_scipy.size # = f_fft_scipy.size = 626 with 8 Hz bin size
    PSD_matrix = np.zeros((window_offsets.size, number_of_bins), dtype="float")
    # calculate all the PSDs
    for index, current_window_offset in enumerate(window_offsets):
        current_window_start_sample = window_start_sample + current_window_offset
        current_window_end_sample = window_duration_samples + current_window_offset
        f_fft_scipy, Sxx_fft_scipy = signal.periodogram(S[current_window_start_sample:current_window_end_sample], fs=Fs, scaling="density")
        PSD_matrix[index] = Sxx_fft_scipy
    # calculate the average PSD
    Sxx_average = np.array([np.mean(freq_bin) for freq_bin in PSD_matrix.T])
    """ 
    # plot the average PSD
    plt.figure()
    plt.grid()
    plt.title(electrode + " average PSD of groups of 80 % overlap 1250 noise segments")
    plt.xlabel("Hz [1/s]")
    plt.ylabel("PSD [pV^2 / Hz]")
    plt.plot(f_fft_scipy, Sxx_average)
    """
    # calculate the max PSD
    Sxx_max = np.array([np.max(freq_bin) for freq_bin in PSD_matrix.T])
    """ 
    # plot the max PSD
    plt.figure()
    plt.grid()
    plt.title(electrode + " max PSD of groups of 80 % overlap 1250 noise segments")
    plt.xlabel("Hz [1/s]")
    plt.ylabel("PSD [pV^2 / Hz]")
    plt.plot(f_fft_scipy, Sxx_max)
    """
    #def make_unscrambler_friendly_list(numpy_array_or_list):
    #    # converts number into string and replaces . comma with , comma
    #    return [ str(number).replace('.',',') for number in numpy_array_or_list ]

    #"""
    # save the average PSD to a file, for use as tresholds on the spark streaming application
    savePathAndFileName = "data/#2_all_noise_segments/PSDNoiseTresholds/" + electrode + "_max_PSD.csv"
    if not os.path.exists("data/#2_all_noise_segments/PSDNoiseTresholds/"):
        os.makedirs("data/#2_all_noise_segments/PSDNoiseTresholds/")
    print("save tresholds to: " + savePathAndFileName)
    np.savetxt(savePathAndFileName, Sxx_max)
    #"""

Minutes of extracted noise on each electrode: 3.68849 [m]
select electrode: 47 (ID=0) [pV]
save tresholds to: data/#2_all_noise_segments/PSDNoiseTresholds/47 (ID=0) [pV]_max_PSD.csv
select electrode: 48 (ID=1) [pV]
save tresholds to: data/#2_all_noise_segments/PSDNoiseTresholds/48 (ID=1) [pV]_max_PSD.csv
select electrode: 46 (ID=2) [pV]
save tresholds to: data/#2_all_noise_segments/PSDNoiseTresholds/46 (ID=2) [pV]_max_PSD.csv
select electrode: 45 (ID=3) [pV]
save tresholds to: data/#2_all_noise_segments/PSDNoiseTresholds/45 (ID=3) [pV]_max_PSD.csv
select electrode: 38 (ID=4) [pV]
save tresholds to: data/#2_all_noise_segments/PSDNoiseTresholds/38 (ID=4) [pV]_max_PSD.csv
select electrode: 37 (ID=5) [pV]
save tresholds to: data/#2_all_noise_segments/PSDNoiseTresholds/37 (ID=5) [pV]_max_PSD.csv
select electrode: 28 (ID=6) [pV]
save tresholds to: data/#2_all_noise_segments/PSDNoiseTresholds/28 (ID=6) [pV]_max_PSD.csv
select electrode: 36 (ID=7) [pV]
save tresholds to: data/#2_all_noise_segm